In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools

import glob

# File to Load (Remember to change these)

dataFiles = glob.glob("Project1\Data\*")
print(dataFiles)
dfs = [pd.read_csv(eachFile) for eachFile in dataFiles ]


dfs = dict()
#'OB': df of OB joins DM_PREV 
#'LTPIA': df of LTPIA joins DM_PREV 
#'INCOME': df of INCOME joins DM_PREV 

dfMapsFiles = dict()

#'DM_PREV': df of DM_PREV
#'OB': df of OB
#'LTPIA': df of LTPIA
#'INCOME': df of INCOME

#drop irrelevant info
dropColsPatterns = {
'DM_PREV': (True, [3,5,6,7,8, 9], 7, "FIPS Codes"),
'OB': (True, [3, 4], 3, "FIPS Codes"),
'LTPIA': (True, [3, 4], 3, "FIPS Codes"),
'INCOME': (False, 1) 
}    

#Data Mungling with CSV files
#calculate drops
def getCalDropCols(series=None, opSet=0, years=9):
    data = pd.DataFrame(series)
    drop = []
    for x in range(years):
        data[x+1] = data[x]+opSet
    for x in data.columns: 
        drop.append(data[x].values.tolist())
    return list( itertools.chain.from_iterable(drop)) 

#cover object to numeric...only applies to Series
def objToFloat(df=None, columns=None):
    print(columns)
    for info in columns:
         df[info] = pd.to_numeric(df[info], errors='coerce')

# clean up data in DM_PREV, OB, LTPIA, INCOME csvs
# drop unnecessary columns and convert to proper dtype
for file, df in zip(dataFiles, dfs):
    for pattern in dropColsPatterns.keys():
        if pattern in file:
            # handles DM_PREV, OB, LTPIA
            if (dropColsPatterns[pattern][0] == True):
            #print(pattern, key)
                array =dropColsPatterns[pattern][1]
                opSet = dropColsPatterns[pattern][2]
                cols = getCalDropCols(array,opSet)
                print(cols)
                columnInfo = list(df.columns)
                df.columns = df.iloc[0]
                comments = df.columns[-1]
                years = [each for each in columnInfo if ("20" in each and not "Un" in each)]
                df = df.drop(df.columns[cols],axis=1)
                comments = df.columns[-1]
                df = df[1:] #create new df
                df['County'] = pd.Series(df['County'].replace(' County', '', regex=True))
                colslabel = ['State', 'FIPS', 'County'] + years
                df.columns = colslabel

                df['Comments'] = comments
                #df.replace("Comments", np.nan, regex=True)
                #print(df.head(3))
                objToFloat(df, years)
                #print(df.head(100).tail(10))
                df = df.dropna()
                df.to_csv(pattern+".csv", na_rep="NaN")
                dfMapsFiles[pattern] = [file, df]
                break
            elif (dropColsPatterns[pattern][0]==False): #handle INCOME
                df.columns = ['Rank', 'State', 'County', 'Per Capita Income', 
                              'Median Household Income', 'Median Family Income', 
                              'Population', '# of households']
                df['County'] = pd.Series(df['County'].replace(' County', '', regex=True))
                df = df.dropna()
                # dollars sign replacement
                df = df[df.columns].replace('[\$,]', '', regex=True)#.astype(int, error=ignore)
                objToFloat(df, df.columns[3:])
                dfMapsFiles[pattern] = [file, df]
                df.to_csv(pattern+".csv", na_rep="NaN")
                break
            #print(df.dtypes)
    
   
#Element 0 = file, 1 = df

df

['Project1\\Data\\DM_PREV_ALL_STATES.csv', 'Project1\\Data\\LTPIA_RANK_ALL_STATES.csv', 'Project1\\Data\\OB_RANK_ALL_STATES.csv', 'Project1\\Data\\WIKI_INCOME_SOURCE.csv']
[3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72]
['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']
[3, 4, 6, 7, 9, 10, 12, 13, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 30, 31]
['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']
[3, 4, 6, 7, 9, 10, 12, 13, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 30, 31]
['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']
Index(['Per Capita Income', 'Median Household Income', 'Median Family Income',
       'Population', '# of households'],
      dtype='object')


,Rank,State,County,Per Capita Income,Median Household Income,Median Family Income,Population,# of households
2,1,New York,New York,62498,69659,84627,1605272,736192
3,2,Virginia,Arlington,62018,103208,139244,214861,94454
4,3,Virginia,Falls Church City,59088,120000,152857,12731,5020
5,4,California,Marin,56791,90839,117357,254643,102912
6,5,Virginia,Alexandria City,54608,85706,107511,143684,65369
7,6,Colorado,Pitkin,51814,72745,93981,17173,7507
8,7,New Mexico,Los Alamos,51044,106686,124979,17979,7590
9,8,Virginia,Fairfax,50532,110292,128596,1101071,389908
10,9,New Jersey,Hunterdon,50349,106143,125828,127047,46816
11,10,Texas,Borden,50042,71607,76250,649,225


In [3]:
dfMapsFiles



{'DM_PREV': ['Project1\\Data\\DM_PREV_ALL_STATES.csv',
              State   FIPS                   County  2004  2005  2006  2007  \
  1         Alabama   1001                  Autauga  10.0  11.1  11.4  11.4   
  2         Alabama   1003                  Baldwin   8.1   8.4   9.1   9.4   
  3         Alabama   1005                  Barbour  10.1  10.7  12.4  13.0   
  4         Alabama   1007                     Bibb   8.9   9.4  10.9  11.6   
  5         Alabama   1009                   Blount   8.3   8.6   9.6  10.5   
  6         Alabama   1011                  Bullock  11.5  12.0  14.0  14.5   
  7         Alabama   1013                   Butler  11.3  11.4  12.8  14.2   
  8         Alabama   1015                  Calhoun   9.8  10.6  11.9  13.5   
  9         Alabama   1017                 Chambers  11.6  12.3  14.2  15.1   
  10        Alabama   1019                 Cherokee   9.6  10.5  12.8  13.3   
  11        Alabama   1021                  Chilton   8.6   9.4   9.5   9.8 

In [ ]:

details = pd.merge(incomeData, df, on=['County','State'])

In [ ]:
year2013 = details.loc[:, ['Rank', 'State', 'County', 'Per Capita Income', 'Median Household Income', 'Median Family Income', 'Population', '2013']]




In [ ]:
year2013.dtypes

In [ ]:
year2013.loc[year2013['State']=='Wyoming'].sort_values(by='2013', ascending=False)